<a href="https://colab.research.google.com/github/taryaksama/data-science/blob/master/Tesselation/TesselationAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Setup

In [ ]:
%cd /content

In [ ]:
#import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Clone GitHub repository
!git clone https://github.com/taryaksama/data-science/Tesselation
%cd Tesselation

# B. Experimental plan

provide a dataframe with experimental plan
- n_exp
- date
- strain
- replica
- file_adress

In [ ]:
%cd data-science/Tesselation
%ls

In [ ]:
path = '.' #address of working directory
data_path = path + '/datasets'

# create a DataFrame with all experiments data
folder_list = [f for f in os.listdir(data_path) if (os.path.isdir(os.path.join(data_path, f)) and f[:6].isdigit())]
exp = pd.DataFrame(folder_list, columns=['folderpath'])
print(exp.head(5))

In [ ]:
exp_date = [exp.folderpath[i][:6] for i in range(len(exp.folderpath))]
exp_strain = [exp.folderpath[i][7:10] for i in range(len(exp.folderpath))]

exp['date'] = exp_date
exp['strain'] = exp_strain
# print(exp['date'].head(5))

# 1. Histogram: Area

## Function

In [ ]:
bin_edges = list(range(100))
bin_edges_norm = list(np.linspace(0,10,101))

def get_area_hist(path, filename, n_strain=0, n_replica=0):
  df = pd.read_csv(path+filename)
  df.columns = ['cellid', 'area', 'area_mean', 'area_norm']

  counts, _, _ = plt.hist(df['area'], bins=bin_edges, density=True)
  counts_norm, _, _ = plt.hist(df['area_norm'], bins=bin_edges_norm, density=True)
  plt.close()

  return counts, counts_norm, np.std(df['area']), np.std(df['area_norm']), df['area_norm'].max(), df['area_norm'].mean()

## Loop for all experiments

In [ ]:
# get histogram from all experiments
exp['area_hist'] = [[] for _ in range(len(exp))]
exp['area_hist_norm'] = [[] for _ in range(len(exp))]
exp['area_hist_std'] = [[] for _ in range(len(exp))]
exp['area_hist_norm_std'] = [[] for _ in range(len(exp))]
exp['area_hist_norm_max'] = [[] for _ in range(len(exp))]
exp['area_hist_norm_max-mean'] = [[] for _ in range(len(exp))]
exp['area_hist_norm_cv'] = [[] for _ in range(len(exp))] # coefficient of variation = std / mean

for n in range(len(exp)):
    #path = path + '/tessellation/'
    path = data_path + '/' +  exp.folderpath[n] + '/tessellation/'
    filename = 'frame_0_voronoi_areas_microns.csv'

    a, b, c, d, e, f = get_area_hist(path, filename)

    exp.at[n, 'area_hist'] = a
    exp.at[n, 'area_hist_norm'] = b
    exp.at[n, 'area_hist_std'] = c
    exp.at[n, 'area_hist_norm_std'] = d
    exp.at[n, 'area_hist_norm_max'] = e
    exp.at[n, 'area_hist_norm_mean'] = f
    exp.at[n, 'area_hist_norm_cv'] = d/f

# 2. Surface Coverage

In [ ]:
# get surface coverage from all experiments
exp['surface_coverage'] = [range(0,len(bin_edges)-1) for _ in range(len(exp))]

for n in range(len(exp)):
  path = 'datasets/' + exp.folderpath[n] + '/surface_coverage/'
  filename = 'surface_coverage_and_density.csv'
  df = pd.read_csv(path+filename)

  exp.at[n, 'surface_coverage'] = float(df.iloc[:,1])

# C. Plots

## Area histograms

In [ ]:
data_plot = pd.DataFrame([], columns=['x', 'y', 'strain'])
#data_plot['x'] = [bin_edges[:-1] for _ in range (len(exp))]
data_plot['x'] = [bin_edges_norm[:-1] for _ in range (len(exp))]
data_plot['y'] = exp['area_hist_norm'] # change depending on column
#data_plot['y'] = exp['area_hist'] # change depending on column
data_plot['strain'] = exp['strain']
data_plot.head(5)

In [ ]:
# Plot histogram

## Print all curves
#for n in range(len(data_plot)):
    #plt.plot(data_plot.at[n, 'x'], data_plot.at[n, 'y'])

## Print all the curves from one strain only
for n in range(len(data_plot)):
    if '232' in data_plot['strain'].iloc[n]:
        plt.plot(data_plot.at[n, 'x'], data_plot.at[n, 'y'])
        plt.yscale('log')
        plt.xlabel('Voronoi Area (µm^2)')
        plt.ylabel('Probability')


#plt.legend(exp['strain']+'_'+exp['date'])
plt.show()

## STD vs. surface coverage

In [ ]:
# --Setup--

# Define colors for each strain
color_map = {'177': 'black', '232': 'deepskyblue', '459': 'darkorange'}

# Define bins and labels for surface coverage
bins = [0, 15, 40, 75]  # Define the edges of the bins
bin_labels = [10, 30, 70]  # Labels for the bins

# Add a new column to assign each point to a bin
exp['surface_coverage_bin'] = pd.cut(exp['surface_coverage'], bins=bins, labels=bin_labels)

In [ ]:
# Create the plot
plt.figure(figsize=(8, 6))  # Optional: Adjust the figure size
for strain in ['177', '232', '459']:
    # Filter the data for the current strain
    strain_data = exp[exp['strain'] == strain]

    # Plot individual points with transparency
    plt.scatter(strain_data['surface_coverage'], strain_data['area_hist_std'],
                color=color_map[strain], alpha=0.5)

    # Group by binned surface coverage and calculate the mean
    grouped = strain_data.groupby('surface_coverage_bin')['area_hist_std'].mean()

    # Plot the averages with a red contour
    plt.scatter(bin_labels, grouped.values, color=color_map[strain], s=100, alpha=1,
                edgecolors='black', linewidths=1.5, label=f'{strain}')

# Add legend and labels
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Surface coverage (%)')
plt.ylabel(r'Standard deviation Voronoi Area ($\mu m^2$)')
plt.xlim([0, 85])
plt.ylim([0, 27])

# Save the figure
plt.savefig('./figures/std_norm_area_plot.png', dpi=300, bbox_inches='tight')  # Save with high resolution

# Show the plot
plt.show()

In [ ]:
# Create the plot
plt.figure(figsize=(8, 6))  # Optional: Adjust the figure size
for strain in ['177', '232', '459']:
    # Filter the data for the current strain
    strain_data = exp[exp['strain'] == strain]

    # Plot individual points with transparency
    plt.scatter(strain_data['surface_coverage'], strain_data['area_hist_norm_cv'],
                color=color_map[strain], alpha=0.5)

    # Group by binned surface coverage and calculate the mean
    grouped = strain_data.groupby('surface_coverage_bin')['area_hist_norm_cv'].mean()

    # Plot the averages with a red contour
    plt.scatter(bin_labels, grouped.values, color=color_map[strain], s=100, alpha=1,
                edgecolors='black', linewidths=1.5, label=f'{strain}')

# Add legend and labels
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Surface coverage (%)')
plt.ylabel(r'Coefficient of Variation Norm. Voronoi Area')
plt.xlim([0, 85])
plt.ylim([0, 0.9])

# Save the figure
plt.savefig('./figures/cv_norm_area_plot.png', dpi=300, bbox_inches='tight')  # Save with high resolution

# Show the plot
plt.show()

In [ ]:
# Create the plot
plt.figure(figsize=(8, 6))  # Optional: Adjust the figure size
for strain in ['177', '232', '459']:
    # Filter the data for the current strain
    strain_data = exp[exp['strain'] == strain]

    # Plot individual points with transparency
    plt.scatter(strain_data['surface_coverage'], strain_data['area_hist_norm_max'],
                color=color_map[strain], alpha=0.3)

    # Group by binned surface coverage and calculate the mean
    grouped = strain_data.groupby('surface_coverage_bin')['area_hist_norm_max'].mean()

    # Plot the averages with a red contour
    plt.scatter(bin_labels, grouped.values, color=color_map[strain], s=150, alpha=1,
                edgecolors='black', linewidths=1.5, label=f'{strain}')

# Add legend and labels
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Surface coverage (%)')
plt.ylabel(r'Max Norm. Voronoi Area')
plt.xlim([0, 85])
plt.ylim([2, 10])
#plt.ylim([2, 50])

# Save the figure
plt.savefig('./figures/max_norm_area_plot.png', dpi=300, bbox_inches='tight')  # Save with high resolution

# Show the plot
plt.show()

In [ ]:
# boxplot for surface coverage
sns.boxplot(exp, x='surface_coverage', y='strain', hue='strain', palette="hls")
sns.stripplot(exp, x='surface_coverage', y='strain', size=4, color=".3")
plt.show()

# boxplot for std(voronoi)
sns.boxplot(exp, x='area_hist_std', y='strain', hue='strain', palette="hls")
sns.stripplot(exp, x='area_hist_std', y='strain', size=4, color=".3")
plt.show()

In [ ]:
# Initialize data
data2plot = exp
data2plot = data2plot.sort_values(by=['strain', 'surface_coverage'], axis=0, ascending=True)
data2plot['bin_edges'] = [bin_edges[:-1] for _ in range (len(data2plot))]
data2plot['bin_edges_norm'] = [bin_edges_norm[:-1] for _ in range (len(data2plot))]

# Bin the surface coverage into categories
bins = [0, 25, 60, 100]
bins_label = ['Low', 'Medium', 'High']
data2plot['surface_coverage_bin'] = pd.cut(data2plot['surface_coverage'], bins=bins, labels=bins_label)

# Create a column for combined category
data2plot['strain_surface_coverage'] = data2plot['strain'] + '_' + data2plot['surface_coverage_bin'].astype(str)

data2plot

1. plot all replicate in gray + mean
- area
- area_norm

one figure per strain, per surface coverage

=> can be a FacetGrid (x=bin, y=counts)
X = surface coverage category
Y = strain

In [ ]:
# Custom function to draw each histogram and an overlay of the mean curve
def plot_records_and_mean(x, y, **kwargs):
  """
  Plots individual records and overlays a mean curve. The function handles both the case
  where there is only one record and the case where there are multiple records. It also
  allows for setting custom x-axis limits via keyword arguments.

  Parameters:
  ----------
  x : list or pandas Series
      List of x-coordinates for the records. If there is a single record, it can be a Series
      of x-values. If there are multiple records, it should be a list of lists/arrays.
  y : list or pandas Series
      List of y-coordinates for the records. If there is a single record, it can be a Series
      of y-values. If there are multiple records, it should be a list of lists/arrays.
  **kwargs : dict, optional
      Additional keyword arguments, including:
      - 'xlim1' (int or float): The lower limit for the x-axis (optional).
      - 'xlim2' (int or float): The upper limit for the x-axis (optional).

  Returns:
  -------
  None
      The function modifies the current matplotlib axis by plotting the individual records
      and overlaying the mean curve for the given data.

  Notes:
  ------
  - If the `x` or `y` inputs contain only one record, the plot will show the data
    without aggregation, and the mean curve will correspond to the given record.
  - If there are multiple records, the function will plot individual records as gray lines
    and overlay a mean curve based on the aggregated data.
  - The x-axis limits can be customized using `xlim1` and `xlim2`. If these are not provided,
    the axis limits will not be adjusted.
  """

  # Extracting xlim from kwargs if it is passed
  xlim1 = kwargs.get('xlim1', None)
  xlim2 = kwargs.get('xlim2', None)
  return_values = kwargs.get('return_values', False)

  ax = plt.gca()

  if len(x)==1: # CASE 1: ONLY 1 DATA
    x_mean = x.to_list()[0]
    y_mean = y.to_list()[0]
    ax.plot(x_mean, y_mean, color='red', linewidth=2, label="Mean Curve")

  else: #CASE 2: MORE THAN 1 DATA
    # Plot individual records
    for x_vals, y_vals in zip(x,y):
      ax.plot(x_vals, y_vals, linestyle='-', color='gray', alpha=0.5)

    # Compute mean data
    x_flattened = [val for sublist in x for val in sublist]
    x_mean = sorted(set(x_flattened))

    # Calculate mean y-values for each unique x
    y_mean = [
        np.mean([y_vals[x_vals.index(pt)] for x_vals, y_vals in zip(x, y) if pt in x_vals])
        for pt in x_mean
    ]

    # Plot the mean curve in blue
    ax.plot(x_mean, y_mean, color='red', linewidth=1, label="Mean Curve")

    # Set x-axis limits if provided
    if xlim1 is not None and xlim2 is not None:
        ax.set_xlim([xlim1, xlim2])

  # Return value of Mean Curve if specified
  if return_values is not False:
    return x_mean, y_mean

In [ ]:
# Initialize the FacetGrid object
g = sns.FacetGrid(data2plot, col='strain_surface_coverage', col_wrap=3, sharex=True, sharey=True)

# Draw on FacetGrid
g.map(plot_records_and_mean, 'bin_edges', 'area_hist', xlim1=0, xlim2=40)

In [ ]:
# Initialize the FacetGrid object
g = sns.FacetGrid(data2plot, col='strain_surface_coverage', col_wrap=3, sharex=True, sharey=True)

# Draw on FacetGrid
g.map(plot_records_and_mean, 'bin_edges_norm', 'area_hist_norm', xlim1=0, xlim2=1)

2. mean histogram with strains combines
- area
- area_norm

one figure per surface coverage

In [ ]:
# Compute the mean area_hist for each strain and surface_coverage_bin
mean_data = data2plot.groupby(['surface_coverage_bin', 'strain']).agg(
    bin_edges=('bin_edges', 'first'),  # Take the first bin_edges for each group
    area_hist=('area_hist', lambda x: np.mean(np.array(list(x.tolist())), axis=0))  # Mean of area_hist
    ).reset_index()

mean_data

In [ ]:
# --per surface coverage, strain overlay--

# Initialize the FacetGrid object with the surface_coverage_bin as columns
g = sns.FacetGrid(mean_data, col='surface_coverage_bin', col_wrap=3, height=4)

mean_data[mean_data['surface_coverage_bin'] =='Low']

# Plot the mean value of area_hist for each strain within each facet
def plot_mean_hist(x, y, **kwargs):
    for n in range(len(x)):
      plt.plot(x.iloc[n], y.iloc[n])
      #plt.yscale('log')

# Map the custom plot function onto the FacetGrid
g.map(plot_mean_hist, 'bin_edges', 'area_hist')
plt.legend(mean_data['strain'].unique())

In [ ]:
# --per strain, surface coverage overlay--

# Initialize the FacetGrid object with the surface_coverage_bin as columns
g = sns.FacetGrid(mean_data, col='strain', col_wrap=5, height=4)

# mean_data[mean_data['surface_coverage_bin'] =='Low']

# Plot the mean value of area_hist for each strain within each facet
def plot_mean_hist(x, y, **kwargs):
    for n in range(len(x)):
      plt.plot(x.iloc[n], y.iloc[n])
      #plt.yscale('log')

# Map the custom plot function onto the FacetGrid
g.map(plot_mean_hist, 'bin_edges', 'area_hist')
plt.legend(mean_data['surface_coverage_bin'].unique())

3. boxplot

- y = STD(area)
- x = strain with 3 sub box (one for each surface coverage)

In [ ]:
# Create the boxplot
sns.boxplot(data=data2plot, x='strain_surface_coverage', y='area_hist_std')
sns.stripplot(data2plot, x='strain_surface_coverage', y='area_hist_std', size=4, color=".3")